In [78]:
import h5py
import pandas as pd
import os


def parse_timestamp(ts):
    ts = str(ts)
    if len(ts) == 17:
        year = int(ts[0:4])
        month = int(ts[4:6])
        day = int(ts[6:8])
        hour = int(ts[8:10])
        minute = int(ts[10:12])
        second = int(ts[12:14])
        millisecond = int(ts[14:17])
        microsecond = millisecond * 1000

        return pd.Timestamp(year, month, day, hour, minute, second, microsecond)
    else:
        return pd.NaT
    
def vwap(df, level=10, _type="all"):
    if _type == "ask":
        pv = df.apply(lambda row: sum([row[f"AskPrice{str(i)}"]*row[f"AskVolume{str(i)}"] for i in range(1,level+1)]),axis=1)
        v = df.apply(lambda row: sum([row[f"AskVolume{str(i)}"] for i in range(1,level+1)]),axis=1)
    if _type == "bid":
        pv = df.apply(lambda row: sum([row[f"BidPrice{str(i)}"]*row[f"BidVolume{str(i)}"] for i in range(1,level+1)]),axis=1)
        v = df.apply(lambda row: sum([row[f"BidVolume{str(i)}"] for i in range(1,level+1)]),axis=1)
    if _type == "all":
        askvwap, askv = vwap(df, level, _type="ask")
        bidvwap, bidv = vwap(df, level, _type="bid")
        pv, v = askvwap*askv + bidvwap*bidv, askv + bidv
    return pv/v, v

class PPL:
    def __init__(self, base_path="./interview"):
        self.base_path = base_path

    def profiler(self, dayprofiler, kwargs={}, id="002521", start_date="20220601", end_date="20221013"):
        dates = list(map(lambda x: x.strftime("%Y%m%d"), pd.date_range(start=start_date, end=end_date).tolist()))
        res = []
        res_dates = []
        for date in dates:
            if os.path.exists(f'{self.base_path}/{id}_{date}.h5'):
                res_dates.append(date)
                with h5py.File(f'{self.base_path}/{id}_{date}.h5', 'r') as f:
                    res.append(dayprofiler(f, **kwargs))
        return pd.concat(res, keys=res_dates)
    
    @staticmethod
    def get_basic(f, freq="1T"):
        df = pd.DataFrame({
            k: f[k][:] for k in f.keys()
        })
        df["MidPrice"] = (df["AskPrice1"] * df["AskVolume1"] + df["BidPrice1"] * df["BidVolume1"]) / (df["AskVolume1"] + df["BidVolume1"])
        df["Time"] = df["DataTime"].apply(parse_timestamp)
        df.set_index("Time", inplace=True)
        resampled = df.resample(freq).agg(
            {
                "MidPrice" : ["first", "last"],
                "Volume" : ["first", "last"],
            }
        )
        res = pd.DataFrame({
            "open" : resampled["MidPrice"]["first"],
            "close" : resampled["MidPrice"]["last"],
            "volume": resampled["Volume"]["last"] - resampled["Volume"]["first"]
        })
        return res
    


In [19]:
pd.set_option("display.max_columns", 100)

In [81]:
f =  h5py.File("./interview/300132_20220616.h5", "r")
print(list(f.keys()))

['AskPrice1', 'AskPrice10', 'AskPrice2', 'AskPrice3', 'AskPrice4', 'AskPrice5', 'AskPrice6', 'AskPrice7', 'AskPrice8', 'AskPrice9', 'AskVolume1', 'AskVolume10', 'AskVolume2', 'AskVolume3', 'AskVolume4', 'AskVolume5', 'AskVolume6', 'AskVolume7', 'AskVolume8', 'AskVolume9', 'AveragePrice', 'BidPrice1', 'BidPrice10', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'BidPrice6', 'BidPrice7', 'BidPrice8', 'BidPrice9', 'BidVolume1', 'BidVolume10', 'BidVolume2', 'BidVolume3', 'BidVolume4', 'BidVolume5', 'BidVolume6', 'BidVolume7', 'BidVolume8', 'BidVolume9', 'ClosePrice', 'DataTime', 'HighestPrice', 'InstrumentID', 'LastPrice', 'LowerLimitPrice', 'LowestPrice', 'Nano', 'OpenPrice', 'PreClosePrice', 'PreSettlPrice', 'SettlPrice', 'TradesCount', 'TradingDay', 'Turnover', 'UpperLimitPrice', 'Volume']


In [5]:
df = pd.DataFrame({
    k: f[k][:] for k in f.keys()
})

In [24]:
df["Time"] = df["DataTime"].apply(parse_timestamp)

In [25]:
df.set_index("Time",inplace=True)


In [50]:
vwap(df, 10, _type="bid")[0]

Time
2022-06-01 09:30:00.030    5.931874
2022-06-01 09:30:00.030    5.943148
2022-06-01 09:30:00.040    5.942739
2022-06-01 09:30:00.050    5.942739
2022-06-01 09:30:00.080    5.942326
                             ...   
2022-06-01 14:56:52.720    6.054720
2022-06-01 14:56:54.830    6.054720
2022-06-01 14:56:54.900    6.054720
2022-06-01 14:56:56.870    6.055362
2022-06-01 14:56:59.770    6.055033
Length: 21353, dtype: float64

In [59]:
df.resample("5s")

In [79]:
ppl = PPL()
ppl.profiler(PPL.get_basic, kwargs={"freq":"1T"})

OSError: Unable to open file (file signature not found)